In [58]:
import os
from pprint import pprint
from dotenv import load_dotenv
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from neo4j.exceptions import ConfigurationError
from langchain_huggingface import HuggingFaceEmbeddings
from src.grag.retrievers.hybrid_cypher.hybrid_cypher import create_hybrid_cypher_retriever_tool

load_dotenv(".env")

True

In [65]:
URI = os.environ["DATABASE_HOST"]
DATABASE = os.environ["DATABASE_SMALL"]
USERNAME = os.environ["DATABASE_USERNAME"]
PASSWORD = os.environ["DATABASE_PASSWORD"]
DATABASE = os.environ["DATABASE_SMALL"]

neo4j_config = {
    "DATABASE_NAME": DATABASE,
    "ARTICLE_VECTOR_INDEX_NAME": os.environ["ARTICLE_VECTOR_INDEX_NAME"],
    "ARTICLE_FULLTEXT_INDEX_NAME": os.environ["ARTICLE_FULLTEXT_INDEX_NAME"],
    "DEFINITION_VECTOR_INDEX_NAME": os.environ["DEFINITION_VECTOR_INDEX_NAME"],
    "DEFINITION_FULLTEXT_INDEX_NAME": os.environ["DEFINITION_FULLTEXT_INDEX_NAME"],
}

# Ngga jadi pakai ini karena Text2Cypher mintannya harus pakai Neo4jGraph
# neo4j_driver = GraphDatabase.driver(uri=URI, auth=(USERNAME, PASSWORD))

neo4j_graph = Neo4jGraph(
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    enhanced_schema=True
)

neo4j_driver = neo4j_graph._driver  # Ambil driver nya kaya gini
embedder_model = HuggingFaceEmbeddings(model_name=os.environ["EMBEDDING_MODEL"])

In [66]:
hybrid_cypher_retriever = create_hybrid_cypher_retriever_tool(
    embedder_model=embedder_model,
    neo4j_driver=neo4j_driver,
    neo4j_config=neo4j_config,
    top_k_initial_article=5,
    total_article_limit=10,
    total_definition_limit=5
)

In [67]:
query = "Apa itu penyelenggara sistem elektronik?"

In [68]:
result = hybrid_cypher_retriever.invoke({"query": query})
print(result)

## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**
--------------------------------------------------------------------------------

Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 30:
(1) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik milik Orang lain dengan cara apa pun.
(2) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik dengan cara apa pun dengan tujuan untuk memperoleh Informasi Elektronik dan/atau Dokumen Elektronik.
(3) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer y^dan/atau Sistem Elektronik dengan cara apa pun dengan melanggar, menerobos, melampaui, atau menjebol sistem pengamanan.

Peraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik, BAB II - PENYELENGGARAAN 

In [69]:
# Agar hasilnya dalam ToolMessage, harus invoke menggunakan schema ToolCall
# https://python.langchain.com/docs/how_to/tool_artifacts/#invoking-the-tool-with-toolcall

result = hybrid_cypher_retriever.invoke(
    {
        "name": "hybrid_cypher_retriever",
        "args": {"query": query},
        "id": "123",  # required
        "type": "tool_call",  # required
    }
)

result.model_dump()

{'content': '## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n--------------------------------------------------------------------------------\n\nUndang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 30:\n(1) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik milik Orang lain dengan cara apa pun.\n(2) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik dengan cara apa pun dengan tujuan untuk memperoleh Informasi Elektronik dan/atau Dokumen Elektronik.\n(3) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer y^dan/atau Sistem Elektronik dengan cara apa pun dengan melanggar, menerobos, melampaui, atau menjebol sistem pengamanan.\n\nPeraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik, BAB 

In [70]:
len(result.content)

7470

In [71]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

checkpointer = MemorySaver()
config = {"configurable": {"thread_id": "1"}}

react_agent = create_react_agent(model=llm, tools=[hybrid_cypher_retriever], checkpointer=checkpointer, version="v2")
print(react_agent.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
          +-------+           
          | agent |           
          +-------+           
         .         .          
       ..           ..        
      .               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


In [72]:
query = "Apa itu penyelenggara sistem elektronik?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query + " Wajib gunakan tool hybrid_cypher_retriever")},
    config=config
)

result["messages"]

[HumanMessage(content='Apa itu penyelenggara sistem elektronik? Wajib gunakan tool hybrid_cypher_retriever', additional_kwargs={}, response_metadata={}, id='b314e39f-c23f-4b76-af4b-35979864337f'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'hybrid_cypher_retriever', 'arguments': '{"query": "Apa itu penyelenggara sistem elektronik?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-65e51cf2-0749-49d3-8c3e-3e7dcfc2fdf4-0', tool_calls=[{'name': 'hybrid_cypher_retriever', 'args': {'query': 'Apa itu penyelenggara sistem elektronik?'}, 'id': '9bcc699c-cfa1-45da-8e08-4a569b3733e0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 50, 'output_tokens': 16, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n----------------------------------------

In [73]:
query = "Bagaimana dengan data pribadi?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query)},
    config=config
)

result["messages"]

[HumanMessage(content='Apa itu penyelenggara sistem elektronik? Wajib gunakan tool hybrid_cypher_retriever', additional_kwargs={}, response_metadata={}, id='b314e39f-c23f-4b76-af4b-35979864337f'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'hybrid_cypher_retriever', 'arguments': '{"query": "Apa itu penyelenggara sistem elektronik?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-65e51cf2-0749-49d3-8c3e-3e7dcfc2fdf4-0', tool_calls=[{'name': 'hybrid_cypher_retriever', 'args': {'query': 'Apa itu penyelenggara sistem elektronik?'}, 'id': '9bcc699c-cfa1-45da-8e08-4a569b3733e0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 50, 'output_tokens': 16, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n----------------------------------------

In [74]:
query = "Tadi definisi penyelenggara sistem elektronik apa? Aku lupa"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query + " Ngga perlu pakai tool call lagi ya")},
    config=config
)

result["messages"]

[HumanMessage(content='Apa itu penyelenggara sistem elektronik? Wajib gunakan tool hybrid_cypher_retriever', additional_kwargs={}, response_metadata={}, id='b314e39f-c23f-4b76-af4b-35979864337f'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'hybrid_cypher_retriever', 'arguments': '{"query": "Apa itu penyelenggara sistem elektronik?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-65e51cf2-0749-49d3-8c3e-3e7dcfc2fdf4-0', tool_calls=[{'name': 'hybrid_cypher_retriever', 'args': {'query': 'Apa itu penyelenggara sistem elektronik?'}, 'id': '9bcc699c-cfa1-45da-8e08-4a569b3733e0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 50, 'output_tokens': 16, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n----------------------------------------

In [135]:
result["messages"][2].model_dump()

{'content': '## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n--------------------------------------------------------------------------------\n\nUndang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 30:\n(1) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik milik Orang lain dengan cara apa pun.\n(2) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik dengan cara apa pun dengan tujuan untuk memperoleh Informasi Elektronik dan/atau Dokumen Elektronik.\n(3) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer y^dan/atau Sistem Elektronik dengan cara apa pun dengan melanggar, menerobos, melampaui, atau menjebol sistem pengamanan.\n\nPeraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik, BAB 

In [80]:
from langchain_core.messages import AIMessage

ai_results = []

for message in result["messages"]:
    if isinstance(message, HumanMessage):
        query = message.content
    if isinstance(message, AIMessage):
        # ai_results.append(message)
        if message.tool_calls:
            ai_results.append({
                "type": "tool_call",
                "id": message.id,
                "query": query,
                "usage": message.usage_metadata
            })
        else:
            ai_results.append({
                "type": "model_response",
                "id": message.id,
                "query": query,
                "response": message.content,
                "usage": message.usage_metadata
            })
                

for ai_result in ai_results:
    pprint(ai_result)
    print()

{'id': 'run-65e51cf2-0749-49d3-8c3e-3e7dcfc2fdf4-0',
 'query': 'Apa itu penyelenggara sistem elektronik? Wajib gunakan tool '
          'hybrid_cypher_retriever',
 'type': 'tool_call',
 'usage': {'input_token_details': {'cache_read': 0},
           'input_tokens': 50,
           'output_tokens': 16,
           'total_tokens': 66}}

{'id': 'run-ebf3c121-68ef-4237-ac1e-1ae988fe175f-0',
 'query': 'Apa itu penyelenggara sistem elektronik? Wajib gunakan tool '
          'hybrid_cypher_retriever',
 'response': 'Penyelenggaraan Sistem Elektronik adalah pemanfaatan Sistem '
             'Elektronik oleh penyelenggara negara, Orang, Badan Usaha, '
             'dan/atau masyarakat. Terdapat juga perbedaan antara '
             'Penyelenggara Sistem Elektronik Lingkup Privat (oleh Orang, '
             'Badan Usaha, dan masyarakat) dan Penyelenggara Sistem Elektronik '
             'Lingkup Publik (oleh Instansi Penyelenggara Negara atau '
             'institusi yang ditunjuk).',
 'type': 'mode

In [77]:
test = llm.invoke("Halo apa yang sedang kamu lakukan? Jawab singkat saja ya")
print(test.content)
print(test.usage_metadata)

Saya sedang menunggu perintahmu.
{'input_tokens': 12, 'output_tokens': 7, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}}


In [78]:
result['messages'][2].model_dump()['artifact']

{'run_time': 0.05475902557373047,
 'is_context_fetched': True,
 'node_ids': [200801011503000,
  201902071501200,
  201902071502300,
  201902071503400,
  201902071502800,
  200801011503700,
  200801011505200,
  202401001503600,
  200801011503600,
  200801011503400,
  201603004400300,
  200801011400600,
  201601019400600,
  201902071400600,
  201902071400500]}

In [79]:
test = llm.invoke(f"Jelaskan data ini: {result['messages'][2].model_dump()['artifact']}")
print(test.content)
print(test.usage_metadata)

Data ini adalah sebuah dictionary (kamus) yang berisi informasi tentang sebuah proses atau eksekusi. Mari kita uraikan setiap elemennya:

*   **`'run_time': 0.05475902557373047`**: Ini menunjukkan waktu yang dibutuhkan untuk menjalankan proses tersebut, dalam satuan detik. Dalam kasus ini, prosesnya berjalan selama sekitar 0.055 detik.

*   **`'is_context_fetched': True`**: Ini adalah nilai boolean (benar atau salah) yang menunjukkan apakah konteks (context) yang diperlukan untuk proses ini berhasil diambil atau tidak. Nilai `True` berarti konteks berhasil diambil. Konteks di sini bisa merujuk pada data, konfigurasi, atau informasi lain yang dibutuhkan proses untuk berjalan dengan benar.

*   **`'node_ids': [200801011503000, 201902071501200, 201902071502300, 201902071503400, 201902071502800, 200801011503700, 200801011505200, 202401001503600, 200801011503600, 200801011503400, 201603004400300, 200801011400600, 201601019400600, 201902071400600, 201902071400500]`**: Ini adalah daftar (list